In [ ]:
!pip install --user --upgrade pinecone-client openai
!pip install --upgrade pinecone-client openai

In [1]:
import os
from openai import OpenAI
from pinecone import Pinecone
import sqlite3
import json
from tqdm import tqdm
import time

In [ ]:
# def alter_database():
#     conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\us_company_info.db')
#     c = conn.cursor()
#     c.execute('''
#               ALTER TABLE companies
#               ADD COLUMN embedding BLOB
#               ''')
#     conn.commit()
#     conn.close()

# # Call this function once to alter the database schema
# alter_database()

In [3]:
def fetch_all_companies():
    conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\us_company_info.db')
    c = conn.cursor()
    c.execute('SELECT * FROM companies')
    companies = c.fetchall()
    conn.close()
    return companies

In [4]:
from dotenv import load_dotenv

load_dotenv(dotenv_path="C:\Users\Admin\PycharmProjects\public-comps\.env")

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

def create_embedding(text):
    response = client.embeddings.create(
      model="text-embedding-ada-002",
      input=text
    )
    return response.data[0].embedding

In [5]:
def update_company_embedding(company_id, embedding):
    conn = sqlite3.connect(r'C:\Users\Admin\PycharmProjects\public-comps\backend\scraping\us_company_info.db')
    c = conn.cursor()
    c.execute('''
              UPDATE companies
              SET embedding = ?
              WHERE id = ?
              ''', (json.dumps(embedding), company_id))
    conn.commit()
    conn.close()

In [ ]:
def extend_database_with_embeddings():
    companies = fetch_all_companies()
    for company in tqdm(companies):
        company_id = company[0]
        longBusinessSummary = company[8]  # Assuming longBusinessSummary is the 9th column
        if longBusinessSummary != 'N/A':
            embedding = create_embedding(longBusinessSummary)
            update_company_embedding(company_id, embedding)
            time.sleep(0.2)  # To avoid hitting the rate limit
        else:
            print(f"Skipping company ID {company_id} due to missing longBusinessSummary")

extend_database_with_embeddings()